# Linear Cryptanalysis

### Linear Approximation

In [1]:
from classes import *
from utils import *
import pandas as pd
import random

### Analysing Cipher Components
We create the linear approximation table and initialise a Substitution Box.

In [2]:
linear_approx_table = {} # save as dictionary before visualisation
sbox = SubBox()

We create linear equations and calculate the probabilities for each equation holding true. For example, take the equation as follows: 

$\quad$ X<sub>2</sub> $\oplus$ X<sub>3</sub> $\oplus$ X<sub>4</sub> = Y<sub>2</sub>

This means we apply a binary mask of [0 1 1 1] to the input bits and a binary mask of [0 1 0 0] to the corresponding output bits, and check to see if the XOR sum of the left hand side of the equation is equal to the XOR sum of the right hand side of the equation.

Each pair of input mask and output mask gives a unique equation. For every input-output pair (16 pairs, where each input gives 1 output), we count the number of times the equation is true.

For a perfectly random substitution box, the distribution should be $\frac{1}{2}$ for all equations. As practically the randomness is not perfect, there is a bias for each equation. The bias refers to the deviation from $\frac{1}{2}$ that the equation holds true.

To calculate the deviation, we take the number of times the equation is true and subtract it by half the number of times it is tested, which in this case is 8. The table is then populated by the deviations. 

In [3]:
# initialise all to -8
for input_mask in range(16):
    for output_mask in range(16):
        linear_approx_table[(input_mask, output_mask)] = -8

# generate combinations of input bits and output bits
for input_mask in range(16):
    for output_mask in range(16):
        input_mask_array = convert_int_to_binary_array(input_mask, 4)
        output_mask_array = convert_int_to_binary_array(output_mask, 4)

        for sbox_input in range(16):
            input_array = convert_int_to_binary_array(sbox_input, 4)
            sbox_output = sbox.sub(sbox_input)
            output_array = convert_int_to_binary_array(sbox_output, 4)

            LHS_array = []
            RHS_array = []

            for i in range(4):
                if input_mask_array[i] == 1:
                    LHS_array.append(input_array[i])
                if output_mask_array[i] == 1:
                    RHS_array.append(output_array[i])

            LHS_result = 0
            RHS_result = 0
            
            for bit in LHS_array:
                LHS_result = LHS_result ^ bit

            for bit in RHS_array:
                RHS_result = RHS_result ^ bit
            
            if LHS_result == RHS_result:
                linear_approx_table[(input_mask, output_mask)] += 1

### Representing the table as a Pandas DataFrame

In [4]:
columns = []
table = {}

for i in range(16):
    columns.append([])

for key, value in linear_approx_table.items():
    columns[key[1]].append(value)

for i in range(16):
    table[i] = columns[i]

linear_approx_table_df = pd.DataFrame(table)
linear_approx_table_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,-2,-2,4,0,2,-2,0,0,2,-2,0,4,2,2,0
2,0,4,0,0,0,4,0,0,-2,2,2,2,2,-2,-2,-2
3,0,-2,2,-4,0,2,2,0,2,4,0,-2,2,0,0,2
4,0,2,2,0,0,2,2,0,2,-4,0,-2,2,4,0,-2
5,0,4,0,0,0,0,0,-4,2,2,-2,2,-2,2,2,2
6,0,2,-2,0,4,-2,2,4,0,2,-2,0,0,2,-2,0
7,0,0,0,0,4,0,-4,0,4,0,4,0,0,0,0,0
8,0,0,-2,-2,-2,2,0,4,0,0,2,2,-2,2,4,0
9,0,-2,4,2,2,0,2,0,0,2,0,2,-2,0,2,-4


The rows of the table correspond to the binary mask of the input, whereas the columns correspond to the binary mask of the output. For example, the equation X<sub>2</sub> $\oplus$ X<sub>3</sub> $\oplus$ X<sub>4</sub> = Y<sub>2</sub> corresponds to row 7 and column 4, which has a bias of +$\frac{4}{16}$.

### Constructing Linear Approximations for the Cryptographic System

#### We use the following approximations of the S-box:  
$\quad$ S<sub>11</sub>: X<sub>2</sub> $\oplus$ X<sub>3</sub> $\oplus$ X<sub>4</sub> = Y<sub>2</sub> 

$\quad$ S<sub>24</sub>: X<sub>4</sub> = Y<sub>3</sub> $\oplus$ Y<sub>4</sub>  

$\quad$ S<sub>31</sub>: X<sub>2</sub> = Y<sub>1</sub> $\oplus$ Y<sub>4</sub>  

$\quad$ S<sub>34</sub>: X<sub>2</sub> = Y<sub>1</sub> $\oplus$ Y<sub>4</sub>  

#### Equation 1:
$\quad$ V<sub>1,2</sub> = U<sub>1,2</sub> $\oplus$ U<sub>1,3</sub> $\oplus$ U<sub>1,4</sub>
= (P<sub>2</sub> $\oplus$ K<sub>1,2</sub>) $\oplus$ (P<sub>3</sub> $\oplus$ K<sub>1,3</sub>) $\oplus$ (P<sub>4</sub> $\oplus$ K<sub>1,4</sub>)

This equation has a probability of $\frac{12}{16}$ with bias $+\frac{1}{4}$.

#### Equation 2:
$\quad$ V<sub>2,15</sub> $\oplus$ V<sub>2,16</sub>= U<sub>2,16</sub>
= K<sub>2,16</sub> $\oplus$ V<sub>1,2</sub>

This equation has a probability of $\frac{12}{16}$ with bias $+\frac{1}{4}$.

#### Equation 3:
Move everything to LHS for equation 2.  
  
$\quad$ V<sub>2,15</sub> $\oplus$ V<sub>2,16</sub> $\oplus$ K<sub>2,16</sub> $\oplus$ V<sub>1,2</sub> = 0

Combining equations 1 and 2.  
  
$\quad$ V<sub>2,15</sub> $\oplus$ V<sub>2,16</sub> $\oplus$ K<sub>2,16</sub> $\oplus$ (P<sub>2</sub> $\oplus$ K<sub>1,2</sub>) $\oplus$ (P<sub>3</sub> $\oplus$ K<sub>1,3</sub>) $\oplus$ (P<sub>4</sub> $\oplus$ K<sub>1,4</sub>) = 0
  
Calculating probabilty.  
  
$\quad$ $\frac{1}{2} + 2(\frac{3}{4}-\frac{1}{2})(\frac{3}{4}-\frac{1}{2}) = \frac{5}{8}$
  
This equation has a probability of $\frac{5}{8}$ with bias $+\frac{1}{8}$.

#### Equation 4:
$\quad$ $(a)$ V<sub>3,1</sub> $\oplus$ V<sub>3,4</sub> = U<sub>3,2</sub> = V<sub>2,16</sub> $\oplus$ K<sub>3,2</sub>  
  
$\quad$ $(b)$ V<sub>3,13</sub> $\oplus$ V<sub>3,16</sub> = U<sub>3,14</sub> = V<sub>2,15</sub> $\oplus$ K<sub>3,14</sub>   
  
Move everything to LHS for equation 4 $(a)$.
  
$\quad$ V<sub>3,1</sub> $\oplus$ V<sub>3,4</sub> $\oplus$ V<sub>2,16</sub> $\oplus$ K<sub>3,2</sub> = 0  
  
Move everything to LHS for equation 4 $(b)$.  

$\quad$ V<sub>3,13</sub> $\oplus$ V<sub>3,16</sub> $\oplus$ V<sub>2,15</sub> $\oplus$ K<sub>3,14</sub> = 0  
  
Combining the equations.  

$\quad$ V<sub>3,1</sub> $\oplus$ V<sub>3,4</sub> $\oplus$ V<sub>2,16</sub> $\oplus$ K<sub>3,2</sub> $\oplus$ V<sub>3,13</sub> $\oplus$ V<sub>3,16</sub> $\oplus$ V<sub>2,15</sub> $\oplus$ K<sub>3,14</sub> = 0

Calculating probabilty.  

$\quad$ $\frac{1}{2} + 2(\frac{1}{4}-\frac{1}{2})(\frac{1}{4}-\frac{1}{2}) = \frac{5}{8}$  
  
This equation has a probability of $\frac{5}{8}$ with bias $+\frac{1}{8}$.

#### Equation 5:
Combining equations 3 and 4.  

$\quad$ V<sub>2,15</sub> $\oplus$ V<sub>2,16</sub> $\oplus$ K<sub>2,16</sub> $\oplus$ (P<sub>2</sub> $\oplus$ K<sub>1,2</sub>) $\oplus$ (P<sub>3</sub> $\oplus$ K<sub>1,3</sub>) $\oplus$ (P<sub>4</sub> $\oplus$ K<sub>1,4</sub>) <br />
$\quad$ $\quad$ $\oplus$ V<sub>3,1</sub> $\oplus$ V<sub>3,4</sub> $\oplus$ V<sub>2,16</sub> $\oplus$ K<sub>3,2</sub> $\oplus$ V<sub>3,13</sub> $\oplus$ V<sub>3,16</sub> $\oplus$ V<sub>2,15</sub> $\oplus$ K<sub>3,14</sub> = 0
  
Simplifying the equation.

$\quad$ V<sub>3,1</sub> $\oplus$ V<sub>3,4</sub> $\oplus$ V<sub>3,13</sub> $\oplus$ V<sub>3,16</sub> $\oplus$ P<sub>2</sub> $\oplus$ P<sub>3</sub> <br />
$\quad$ $\quad$ $\oplus$ P<sub>4</sub> $\oplus$ K<sub>1,2</sub> $\oplus$ K<sub>1,3</sub> $\oplus$ K<sub>1,4</sub> $\oplus$ K<sub>2,16</sub> $\oplus$  K<sub>3,2</sub> $\oplus$ K<sub>3,14</sub> = 0
  
Note that  

$\quad$ $(a)$ U<sub>4,7</sub> = K<sub>4,7</sub> $\oplus$ V<sub>3,1</sub> $\Rightarrow$ V<sub>3,1</sub> = U<sub>4,7</sub> $\oplus$ K<sub>4,7</sub>  

$\quad$ $(b)$ U<sub>4,5</sub> = K<sub>4,5</sub> $\oplus$ V<sub>3,4</sub> $\Rightarrow$ V<sub>3,4</sub> = U<sub>4,5</sub> $\oplus$ K<sub>4,5</sub>  

$\quad$ $(c)$ U<sub>4,6</sub> = K<sub>4,6</sub> $\oplus$ V<sub>3,13</sub> $\Rightarrow$  V<sub>3,13</sub> = U<sub>4,6</sub> $\oplus$ K<sub>4,6</sub>  

$\quad$ $(d)$ U<sub>4,2</sub> = K<sub>4,2</sub> $\oplus$ V<sub>3,16</sub> $\Rightarrow$ V<sub>3,16</sub> = U<sub>4,2</sub> $\oplus$ K<sub>4,2</sub>  
  
$\quad$ $\therefore$ U<sub>4,7</sub> $\oplus$ K<sub>4,7</sub> $\oplus$ U<sub>4,5</sub> $\oplus$ K<sub>4,5</sub> $\oplus$ U<sub>4,6</sub> $\oplus$ K<sub>4,6</sub> $\oplus$ U<sub>4,2</sub> $\oplus$ K<sub>4,2</sub> $\oplus$ P<sub>2</sub> <br />
$\quad$ $\quad$ $\oplus$ P<sub>3</sub> $\oplus$ P<sub>4</sub> $\oplus$ K<sub>1,2</sub> $\oplus$ K<sub>1,3</sub> $\oplus$ K<sub>1,4</sub> $\oplus$ K<sub>2,16</sub> $\oplus$  K<sub>3,2</sub> $\oplus$ K<sub>3,14</sub> = 0  
  
Let $\sum_{k}$ = K<sub>4,7</sub> $\oplus$ K<sub>4,5</sub> $\oplus$ K<sub>4,6</sub> $\oplus$ K<sub>4,2</sub> $\oplus$ K<sub>1,2</sub> $\oplus$ K<sub>1,3</sub> $\oplus$ K<sub>1,4</sub> $\oplus$ K<sub>2,16</sub> $\oplus$  K<sub>3,2</sub> $\oplus$ K<sub>3,14</sub>  
  
We get the following equation. 

$\quad$ U<sub>4,7</sub> $\oplus$  U<sub>4,5</sub> $\oplus$ U<sub>4,6</sub> $\oplus$  U<sub>4,2</sub> $\oplus$ P<sub>2</sub> $\oplus$ P<sub>3</sub> $\oplus$ P<sub>4</sub> $\oplus$ $\sum_{k}$ = 0
  
Calculating probabilty. 

$\quad$ $\frac{1}{2} + 2^3(\frac{12}{16}-\frac{1}{2})(\frac{12}{16}-\frac{1}{2})(\frac{4}{16}-\frac{1}{2})(\frac{4}{16}-\frac{1}{2}) = \frac{17}{32}$  
  
Depending on whether  $\sum_{k}$ is 0 or 1, the probability is $\frac{17}{32}$ or $\frac{15}{32}$ (with bias of magnitude $\frac{1}{32}$)

### Generating Plaintexts and Corresponding Ciphertexts

We will generate 10000 plaintexts and their corresponding ciphertexts.

In [5]:
spn = SPN()
master_key = 42069

In [6]:
plaintext_list = []
ciphertext_list = []
num_plaintext = 10000

while len(plaintext_list) < num_plaintext:
    plaintext = random.randint(0, 65535)
    if plaintext not in plaintext_list:
        plaintext_list.append(plaintext)
        ciphertext = spn.encrypt(plaintext, master_key)
        ciphertext_list.append(ciphertext)

For the purposes of this demonstration, we will be carrying out a known plaintext attack. In assuming the role of the attacker, we will have the 10000 plaintexts and corresponding ciphertexts.

In [7]:
keyspace = {}
for i in range(256):
    keyspace[i] = 0

for key in keyspace.keys():
    for index, ciphertext in enumerate(ciphertext_list):
        key_binary_array = convert_int_to_binary_array(key, 8)
        # pad key to 16 bits
        key_binary_array.extend([0 for i in range(16 - len(key_binary_array))])
        ciphertext_binary_array = convert_int_to_binary_array(ciphertext)

        # reverse xor
        reverse_xor_result = convert_binary_array_to_int(xor_binary_arrays(key_binary_array, ciphertext_binary_array))

        partial_ciphertext_array = split_bits_to_4_bit_int(reverse_xor_result)[:2]

        # reverse sub
        reverse_sub_result = [sbox.unsub(partial_ciphertext_array[0]), sbox.unsub(partial_ciphertext_array[1])]

        reverse_sub_array = convert_int_to_binary_array(reverse_sub_result[0], 4)
        reverse_sub_array.extend(convert_int_to_binary_array(reverse_sub_result[1], 4))
        
        # xor result
        result = reverse_sub_array[1]
        result = result ^ reverse_sub_array[4]
        result = result ^ reverse_sub_array[5]
        result = result ^ reverse_sub_array[6]

        plaintext_binary_array = convert_int_to_binary_array(plaintext_list[index])

        result = result ^ plaintext_binary_array[1]
        result = result ^ plaintext_binary_array[2]
        result = result ^ plaintext_binary_array[3]

        if result == 0:
            keyspace[key] += 1

In [8]:
print(max(keyspace.values()) == min(keyspace.values()))

False


In [9]:
max(keyspace.values())

5330

In [24]:
subkeys = []
bias = []

for key, value in keyspace.items():
    subkeys.append(key)
    bias.append(float(f'{abs(value-5000)/10000:.4f}'))

linear_attack_bias_df = pd.DataFrame({'subkey':subkeys, 'bias':bias})
# pd.set_option('display.max_rows', None)
linear_attack_bias_df.style.hide_index()

subkey,bias
0,0.006900
1,0.019200
2,0.004700
3,0.011700
4,0.012700
5,0.000600
6,0.004200
7,0.011200
8,0.017200
9,0.010300


In [25]:
print(linear_attack_bias_df['bias'].max())
print(linear_attack_bias_df['bias'].idxmax())

0.0356
173
